# Ensemble Learning Regressor

## Mục tiêu

- Xây dựng mô hình Ensemble Learning kết hợp các base learners đã huấn luyện từ các notebooks trước
- Sử dụng hai phương pháp Ensemble Learning là Voting Ensemble và Stacking Ensemble
- Đánh giá hiệu suất trên tập train bằng: MAE, RMSE, R²
- Lưu mô hình Ensemble

## Giới thiệu

Voting Ensemble kết hợp dự đoán của nhiều mô hình bằng weighted average:
- Giảm variance (lỗi ngẫu nhiên)
- Giảm bias (lỗi hệ thống)
- Tăng độ ổn định và tổng quát hóa

## Bước 1 - Import các thư viện cần thiết

In [1]:
from pathlib import Path
import joblib
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
import numpy as np


## Bước 2 - Đọc dữ liệu đã tiền xử lý

In [2]:
# Đọc dữ liệu
train_df = pd.read_csv('../data/processed/train.csv')

print("THÔNG TIN DỮ LIỆU")
print("="*60)
print(f"Kích thước tập train: {train_df.shape}")
print("\n5 dòng đầu tiên của tập train:")
train_df.head()

THÔNG TIN DỮ LIỆU
Kích thước tập train: (3124, 13)

5 dòng đầu tiên của tập train:


,country_name,country_code,year,population,pop_growth,life_expectancy,gdp_per_capita,gdp_growth,sanitation,electricity,water_access,co2_emissions,labor_force
0,Denmark,DNK,2017,-0.203264,-0.418953,81.102439,1.682694,-0.047741,1.673510,0.642797,0.754689,0.118599,0.069223
1,"Korea, Dem. People's Rep.",PRK,2017,-0.056076,-0.530921,73.034000,-0.412271,0.028651,-0.245951,-1.294730,0.439985,-0.216848,2.087276
2,Madagascar,MDG,2008,-0.091998,1.007790,61.992000,-0.607170,0.547296,-1.651029,-2.194047,-2.540432,-0.528445,2.542268
3,Greece,GRC,2018,-0.166799,-0.945727,81.787805,0.170491,-0.209157,1.356075,0.642797,0.754689,0.161554,-1.033944
4,South Sudan,SSD,2019,-0.169071,1.000983,58.129000,-0.412271,0.028651,-1.375450,-2.593358,-2.691194,-0.278341,1.279234


## Bước 3 - Chuẩn bị dữ liệu

In [3]:
# Định nghĩa các cột dùng để dự đoán
feature_cols = [col for col in train_df.columns
                if col not in ['life_expectancy', 'country_name', 'country_code']]

# Tách X và y
X_train = train_df[feature_cols]
y_train = train_df['life_expectancy']

print("THÔNG TIN CÁC TẬP DỮ LIỆU")
print("="*60)
print(f"Số lượng đặc trưng: {len(feature_cols)}") 
print(f"Kích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")

THÔNG TIN CÁC TẬP DỮ LIỆU
Số lượng đặc trưng: 10
Kích thước X_train: (3124, 10)
Kích thước y_train: (3124,)


## Bước 4 - Load các mô hình cơ sở (Base Learners)

In [4]:
BASE_DIR = Path("../models")

# ===== 1. Linear Models =====
linear_reg = joblib.load(BASE_DIR / "1_linear_regression" / "linear.pkl")
ridge_reg  = joblib.load(BASE_DIR / "1_linear_regression" / "ridge.pkl")
lasso_reg  = joblib.load(BASE_DIR / "1_linear_regression" / "lasso.pkl")

# ===== 2. SVM =====
linear_svr = joblib.load(BASE_DIR / "2_SVM_regression" / "linear_svr.pkl")
svr_rbf    = joblib.load(BASE_DIR / "2_SVM_regression" / "svr_rbf.pkl")

# ===== 3. Decision Tree =====
decision_tree = joblib.load(BASE_DIR / "3_decision_tree" / "decision_tree.pkl")

# ===== 4. Random Forest =====
random_forest = joblib.load(BASE_DIR / "4_random_forest" / "random_forest.pkl.gz")

# ===== 5. GBM =====
gbm = joblib.load(BASE_DIR / "5_GBM" / "GBM.pkl.gz")

# ===== 6. LightGBM =====
lightgbm = joblib.load(BASE_DIR / "6_lightgbm" / "lightgbm.pkl")    

# ===== 7. XGBoost =====
xgboost = joblib.load(BASE_DIR / "7_xgboost" / "xgboost.pkl.gz")

# ===== 8. CatBoost =====
catboost = joblib.load(BASE_DIR / "8_catboost" / "catboost.pkl.gz")

print("Tất cả các mô hình đã được load thành công!")

Tất cả các mô hình đã được load thành công!


# I. Voting Ensemble

## Bước 1 - Xây dựng Voting Ensemble với Weighted Average

### 1.1. Gom các base model vào dictionary

In [5]:
# Gom models vào dictionary
base_models = {
    'Linear': linear_reg,
    'Ridge': ridge_reg,
    'Lasso': lasso_reg,
    'LinearSVR': linear_svr,
    'SVR_RBF': svr_rbf,
    'DecisionTree': decision_tree,
    'RandomForest': random_forest,
    'GBM': gbm,
    'XGBoost': xgboost,
    'LightGBM': lightgbm,
    'CatBoost': catboost
}

### 1.2. Tính toán R², MAE, RMSE của từng base model

In [6]:
# Tính metrics cho từng model
print("="*70)
print("ĐÁNH GIÁ CÁC BASE LEARNERS TRÊN TẬP TRAIN")
print("="*70)

model_scores = {}

for model_name, model in base_models.items():
    y_pred = model.predict(X_train)
    mae = mean_absolute_error(y_train, y_pred)
    rmse = root_mean_squared_error(y_train, y_pred)
    r2 = r2_score(y_train, y_pred)
    
    model_scores[model_name] = {'MAE': mae, 'RMSE': rmse, 'R2': r2}
    
    print(f"\n{model_name:<15} | MAE: {mae:7.4f} | RMSE: {rmse:7.4f} | R²: {r2:7.4f}")

print("\n" + "="*70)

ĐÁNH GIÁ CÁC BASE LEARNERS TRÊN TẬP TRAIN

Linear          | MAE:  3.1661 | RMSE:  4.2541 | R²:  0.7670

Ridge           | MAE:  3.1682 | RMSE:  4.2544 | R²:  0.7670

Lasso           | MAE:  3.1671 | RMSE:  4.2551 | R²:  0.7669

LinearSVR       | MAE:  3.1586 | RMSE:  4.3049 | R²:  0.7614

SVR_RBF         | MAE:  1.4064 | RMSE:  2.4844 | R²:  0.9205

DecisionTree    | MAE:  1.6345 | RMSE:  2.2304 | R²:  0.9360

RandomForest    | MAE:  0.2001 | RMSE:  0.3805 | R²:  0.9981

GBM             | MAE:  0.1569 | RMSE:  0.2154 | R²:  0.9994

XGBoost         | MAE:  0.0971 | RMSE:  0.1298 | R²:  0.9998

LightGBM        | MAE:  1.4403 | RMSE:  2.2144 | R²:  0.9369

CatBoost        | MAE:  1.4070 | RMSE:  2.0891 | R²:  0.9438



### 1.3. Tính trọng số và Weighted Average Prediction

Tại bước này, ta sẽ áp dụng MAE để đánh trọng số cho các base learners.

1. MAE (Mean Absolute Error) được lựa chọn vì những lý do sau:

- MAE ổn định, ít nhạy cảm với outliers.
   - Dữ liệu được sử dụng có sự chênh lệch lớn về tuổi thọ giữa các quốc gia top đầu và các quốc gia top dưới. Do đó việc áp dụng MAE là phù hợp.

- MAE đo sai số trực tiếp theo đơn vị tuổi thọ.
   - Rất dễ diễn giải, trực quan, phù hợp với bài toán sức khỏe dân số.
   - Ví dụ: MAE = 1.8 tức là mô hình sai trung bình 1.8 năm tuổi thọ.

2. Các metrics còn lại (R2, RMSE) đều không phù hợp do:
- $R^2$ không phản ánh sai số dự đoán thật, chỉ có thể giải thích phương sai của phân phối.
- RMSE cực nhạy với các outliers, có thể tăng mạnh đối với dữ liệu tuổi thọ hiện có.


3. Trọng số nghịch đảo với MAE:
   $$w_i = \frac{1/MAE_i}{\sum_j (1/MAE_j)}$$
   - MAE nhỏ → 1/MAE lớn → trọng số cao (mô hình tốt hơn)
   - MAE lớn → 1/MAE nhỏ → trọng số thấp (mô hình kém hơn)
   - Tự động chuẩn hóa: $\sum w_i = 1$

In [7]:
# Tính trọng số dựa trên MAE (nghịch đảo)
mae_values = {name: scores['MAE'] for name, scores in model_scores.items()}
inverse_mae = {name: 1.0 / mae for name, mae in mae_values.items()}
total_inverse_mae = sum(inverse_mae.values())
weights = {name: inv_mae / total_inverse_mae for name, inv_mae in inverse_mae.items()}

print("TRỌNG SỐ CỦA MỖI MODEL (dựa trên MAE nghịch đảo)")
print("="*70)
print(f"{'Model':<15} | {'MAE':<10} | {'1/MAE':<10} | {'Trọng số':<10} | {'%':<8}") 
print("-"*70)

for model_name in sorted(weights.keys(), key=lambda x: weights[x], reverse=True):
    mae = mae_values[model_name]
    inv = inverse_mae[model_name]
    w = weights[model_name]
    pct = w * 100
    print(f"{model_name:<15} | {mae:<10.4f} | {inv:<10.4f} | {w:<10.4f} | {pct:<8.2f}%")

TRỌNG SỐ CỦA MỖI MODEL (dựa trên MAE nghịch đảo)
Model           | MAE        | 1/MAE      | Trọng số   | %       
----------------------------------------------------------------------
XGBoost         | 0.0971     | 10.3018    | 0.4014     | 40.14   %
GBM             | 0.1569     | 6.3734     | 0.2483     | 24.83   %
RandomForest    | 0.2001     | 4.9978     | 0.1947     | 19.47   %
SVR_RBF         | 1.4064     | 0.7110     | 0.0277     | 2.77    %
CatBoost        | 1.4070     | 0.7107     | 0.0277     | 2.77    %
LightGBM        | 1.4403     | 0.6943     | 0.0271     | 2.71    %
DecisionTree    | 1.6345     | 0.6118     | 0.0238     | 2.38    %
LinearSVR       | 3.1586     | 0.3166     | 0.0123     | 1.23    %
Linear          | 3.1661     | 0.3158     | 0.0123     | 1.23    %
Lasso           | 3.1671     | 0.3157     | 0.0123     | 1.23    %
Ridge           | 3.1682     | 0.3156     | 0.0123     | 1.23    %


In [8]:
# Tính Weighted Average Prediction
y_pred_voting = np.zeros(len(y_train))

for model_name, weight in weights.items():
    model = base_models[model_name]
    y_pred_model = model.predict(X_train)
    y_pred_voting += weight * y_pred_model

print("Tính toán Weighted Average Prediction hoàn tất")

Tính toán Weighted Average Prediction hoàn tất


## Bước 2 - Đánh giá Voting Ensemble

In [9]:
# Tính metrics cho Voting Ensemble
mae_voting = mean_absolute_error(y_train, y_pred_voting)
rmse_voting = root_mean_squared_error(y_train, y_pred_voting)
r2_voting = r2_score(y_train, y_pred_voting)

print("KẾT QUẢ VOTING ENSEMBLE")
print("="*70)
print(f"MAE  : {mae_voting:7.4f}")
print(f"RMSE : {rmse_voting:7.4f}")
print(f"R²   : {r2_voting:7.4f}\n")

print("SO SÁNH VỚI CÁC BASE LEARNERS")
print("="*70)
print(f"{'Model':<15} | {'MAE':<10} | {'RMSE':<10} | {'R²':<10}")
print("-"*70)

# In từng model
for model_name in sorted(model_scores.keys(), key=lambda x: model_scores[x]['R2'], reverse=True):
    mae = model_scores[model_name]['MAE']
    rmse = model_scores[model_name]['RMSE']
    r2 = model_scores[model_name]['R2']
    print(f"{model_name:<15} | {mae:<10.4f} | {rmse:<10.4f} | {r2:<10.4f}")

print("-"*70)
print(f"{'VOTING':<15} | {mae_voting:<10.4f} | {rmse_voting:<10.4f} | {r2_voting:<10.4f}")


KẾT QUẢ VOTING ENSEMBLE
MAE  :  0.3305
RMSE :  0.4593
R²   :  0.9973

SO SÁNH VỚI CÁC BASE LEARNERS
Model           | MAE        | RMSE       | R²        
----------------------------------------------------------------------
XGBoost         | 0.0971     | 0.1298     | 0.9998    
GBM             | 0.1569     | 0.2154     | 0.9994    
RandomForest    | 0.2001     | 0.3805     | 0.9981    
CatBoost        | 1.4070     | 2.0891     | 0.9438    
LightGBM        | 1.4403     | 2.2144     | 0.9369    
DecisionTree    | 1.6345     | 2.2304     | 0.9360    
SVR_RBF         | 1.4064     | 2.4844     | 0.9205    
Linear          | 3.1661     | 4.2541     | 0.7670    
Ridge           | 3.1682     | 4.2544     | 0.7670    
Lasso           | 3.1671     | 4.2551     | 0.7669    
LinearSVR       | 3.1586     | 4.3049     | 0.7614    
----------------------------------------------------------------------
VOTING          | 0.3305     | 0.4593     | 0.9973    


## Bước 3 - Lưu Voting Ensemble Model

In [12]:
# Tạo dictionary lưu Voting Ensemble
voting_ensemble_data = {
    'base_models': base_models,
    'weights': weights,
    'model_scores': model_scores,
    'metrics': {
        'MAE': mae_voting,
        'RMSE': rmse_voting,
        'R2': r2_voting
    },
    'feature_names': feature_cols
}

# Lưu mô hình
os.makedirs('../models/9_ensemble_learning', exist_ok=True)
voting_path = '../models/9_ensemble_learning/voting_ensemble.pkl.gz'

joblib.dump(voting_ensemble_data, voting_path, compress=('gzip', 9))

['../models/9_ensemble_learning/voting_ensemble.pkl.gz']

## Kết luận

**Tổng kết:**
1. Mô hình Ensemble Learning đã được xây dựng thành công.
2. Sử dụng Voting Ensemble với weighted average (trọng số dựa trên MAE score)
3. Mô hình được đánh giá dựa trên 3 metrics trên tập train: MAE, RMSE và R2


# II. Stacking Ensemble

## Bước 1: Xây dựng Stacking Ensemble

In [13]:
from sklearn.ensemble import StackingRegressor


### 1.1. Gom các base model vào dictionary

In [14]:
base_models_2 = {
    'SVR_RBF': svr_rbf,
    'RandomForest': random_forest,
    'LightGBM': lightgbm,
    'XGBoost': xgboost,
    'GBM': gbm
}

### 1.2. Tính toán R², MAE, RMSE của từng base model

In [15]:
# Tính metrics cho từng model
print("="*70)
print("ĐÁNH GIÁ CÁC BASE LEARNERS TRÊN TẬP TRAIN")
print("="*70)

model_scores_2 = {}

for model_name, model in base_models_2.items():
    y_pred = model.predict(X_train)
    mae = mean_absolute_error(y_train, y_pred)
    rmse = root_mean_squared_error(y_train, y_pred)
    r2 = r2_score(y_train, y_pred)
    
    model_scores_2[model_name] = {'MAE': mae, 'RMSE': rmse, 'R2': r2}
    
    print(f"\n{model_name:<15} | MAE: {mae:7.4f} | RMSE: {rmse:7.4f} | R²: {r2:7.4f}")

print("\n" + "="*70)

ĐÁNH GIÁ CÁC BASE LEARNERS TRÊN TẬP TRAIN

SVR_RBF         | MAE:  1.4064 | RMSE:  2.4844 | R²:  0.9205

RandomForest    | MAE:  0.2001 | RMSE:  0.3805 | R²:  0.9981

LightGBM        | MAE:  1.4403 | RMSE:  2.2144 | R²:  0.9369

XGBoost         | MAE:  0.0971 | RMSE:  0.1298 | R²:  0.9998

GBM             | MAE:  0.1569 | RMSE:  0.2154 | R²:  0.9994



### 1.3. Xây dựng mô hình Stacking Ensemble
- Sử dụng 5 base model: SVR_RBF, Random Forest, Lightgbm, GBM, Xgboost
- Model tầng 2 (meta-model):  Ridge

In [16]:
stack = StackingRegressor(
    estimators=[(name, model) for name, model in base_models_2.items()],
    final_estimator=ridge_reg,
    cv=5,
)


### 1.4. Huấn luyện mô hình

In [18]:
stack.fit(X_train, y_train)
y_pred_stack = stack.predict(X_train)

## Bước 2 - Đánh giá Stacking Ensemble

In [20]:
# Tính metrics cho Stacking Ensemble
mae_stack = mean_absolute_error(y_train, y_pred_stack)
rmse_stack = root_mean_squared_error(y_train, y_pred_stack)
r2_stack = r2_score(y_train, y_pred_stack)

print("KẾT QUẢ STACKING ENSEMBLE")
print("="*70)
print(f"MAE  : {mae_stack:7.4f}")
print(f"RMSE : {rmse_stack:7.4f}")
print(f"R²   : {r2_stack:7.4f}\n")
print("SO SÁNH VỚI CÁC BASE LEARNERS")
print("="*70)
print(f"{'Model':<15} | {'MAE':<10} | {'RMSE':<10} | {'R²':<10}")
print("-"*70)

# In từng model
for model_name in sorted(model_scores.keys(), key=lambda x: model_scores[x]['R2'], reverse=True):
    mae = model_scores[model_name]['MAE']
    rmse = model_scores[model_name]['RMSE']
    r2 = model_scores[model_name]['R2']
    print(f"{model_name:<15} | {mae:<10.4f} | {rmse:<10.4f} | {r2:<10.4f}")

print("-"*70)
# print(f"{'VOTING':<15} | {mae_voting:<10.4f} | {rmse_voting:<10.4f} | {r2_voting:<10.4f}")
print(f"{'STACK':<15} | {mae_stack:<10.4f} | {rmse_stack:<10.4f} | {r2_stack:<10.4f}")


KẾT QUẢ STACKING ENSEMBLE
MAE  :  0.1780
RMSE :  0.2488
R²   :  0.9992

SO SÁNH VỚI CÁC BASE LEARNERS
Model           | MAE        | RMSE       | R²        
----------------------------------------------------------------------
XGBoost         | 0.0971     | 0.1298     | 0.9998    
GBM             | 0.1569     | 0.2154     | 0.9994    
RandomForest    | 0.2001     | 0.3805     | 0.9981    
CatBoost        | 1.4070     | 2.0891     | 0.9438    
LightGBM        | 1.4403     | 2.2144     | 0.9369    
DecisionTree    | 1.6345     | 2.2304     | 0.9360    
SVR_RBF         | 1.4064     | 2.4844     | 0.9205    
Linear          | 3.1661     | 4.2541     | 0.7670    
Ridge           | 3.1682     | 4.2544     | 0.7670    
Lasso           | 3.1671     | 4.2551     | 0.7669    
LinearSVR       | 3.1586     | 4.3049     | 0.7614    
----------------------------------------------------------------------
STACK           | 0.1780     | 0.2488     | 0.9992    


## Bước 3 - Lưu Stacking Ensemble Model

In [21]:
# Tạo dictionary lưu Voting Ensemble
stacking_ensemble_data = {
    'stacking_model': stack,
    'base_models': base_models_2,
    'model_scores': model_scores_2,
    'metrics': {
        'MAE': mae_stack,
        'RMSE': rmse_stack,
        'R2': r2_stack
    },
    'feature_names': feature_cols
}

# Lưu mô hình
stacking_path = '../models/9_ensemble_learning/stacking_ensemble.pkl.gz'

joblib.dump(stacking_ensemble_data, stacking_path, compress=('gzip', 9))

['../models/9_ensemble_learning/stacking_ensemble.pkl.gz']

## Kết luận

**Tổng kết:**
1. Mô hình Ensemble Learning đã được xây dựng thành công.
2. Sử dụng Stacking Ensemble với 5 base model và 1 meta model
3. Mô hình được đánh giá dựa trên 3 metrics trên tập train: MAE, RMSE và R2
